### Importer

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


sys.meta_path.append(NotebookFinder())

### Import

In [2]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset
import datasplit
import models
import trainvaltest
import utils

### GPU

In [3]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [4]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed', '808', 'BD2', 'BMF', 'DPL',
                 'DS1', 'FFC', 'MGS', 'OD1', 'RAT', 'RBM', 'SD1', 'TS9', 'VTB']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [5]:
len(dataset)

39936

### TRAIN FxNET

In [6]:
# model
fxnet = models.FxNet().to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

In [7]:
print(fxnet)

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=14, bias=True)
)


In [8]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainvaltest.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainvaltest.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        epoch=epoch, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainvaltest.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        epoch=epoch, 
        device=device
    )
    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

0.0012
Train Epoch: 46	[58000/79228 (73%)]	Total Loss: 69.7318	Avg Loss: 0.0012
Train Epoch: 46	[59000/79228 (74%)]	Total Loss: 70.8550	Avg Loss: 0.0012
Train Epoch: 46	[60000/79228 (76%)]	Total Loss: 72.0533	Avg Loss: 0.0012
Train Epoch: 46	[61000/79228 (77%)]	Total Loss: 73.1320	Avg Loss: 0.0012
Train Epoch: 46	[62000/79228 (78%)]	Total Loss: 74.4023	Avg Loss: 0.0012
Train Epoch: 46	[63000/79228 (79%)]	Total Loss: 75.5204	Avg Loss: 0.0012
Train Epoch: 46	[64000/79228 (81%)]	Total Loss: 76.8072	Avg Loss: 0.0012
Train Epoch: 46	[65000/79228 (82%)]	Total Loss: 78.0436	Avg Loss: 0.0012
Train Epoch: 46	[66000/79228 (83%)]	Total Loss: 79.3618	Avg Loss: 0.0012
Train Epoch: 46	[67000/79228 (84%)]	Total Loss: 80.6868	Avg Loss: 0.0012
Train Epoch: 46	[68000/79228 (86%)]	Total Loss: 81.7708	Avg Loss: 0.0012
Train Epoch: 46	[69000/79228 (87%)]	Total Loss: 83.0891	Avg Loss: 0.0012
Train Epoch: 46	[70000/79228 (88%)]	Total Loss: 84.6155	Avg Loss: 0.0012
Train Epoch: 46	[71000/79228 (90%)]	Total Lo

In [10]:
# SAVE
models_folder = 'saved_models'
model_name = '20200828_fxnet'
results_folder = 'saved_results'

In [12]:
# SAVE MODEL
torch.save(fxnet, '%s/%s' % (models_folder, model_name))

In [14]:
# SAVE RESULTS
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
all_train_results_npy = np.array(all_train_results)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
all_val_results_npy = np.array(all_val_results)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
all_test_results_npy = np.array(all_test_results)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_results')), arr=all_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_results')), arr=all_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_results')), arr=all_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'fx_labels')), arr=fx_labels_npy)

### TRAIN SetNET

In [7]:
# model
setnet = models.SettingsNet().to(device)
# optimizer
optimizer_setnet = optim.Adam(setnet.parameters(), lr=0.001)
# loss function
loss_func_setnet = nn.MSELoss(reduction='mean')

In [8]:
print(setnet)

SettingsNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=6, bias=True)
)


In [9]:
# TRAIN and TEST SetNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainvaltest.train_settings_net(
        model=setnet,
        optimizer=optimizer_setnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainvaltest.val_settings_net(
        model=setnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet,
        device=device
    )
    
    test_loss, test_correct, test_results = trainvaltest.test_settings_net(
        model=setnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        epoch=epoch,
        loss_function=loss_func_setnet,
        device=device
    )
    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

oss: 0.0561	Avg Loss: 0.0000
Train Epoch: 41	[9000/28753 (31%)]	Total Loss: 0.0621	Avg Loss: 0.0000
Train Epoch: 41	[10000/28753 (35%)]	Total Loss: 0.0722	Avg Loss: 0.0000
Train Epoch: 41	[11000/28753 (38%)]	Total Loss: 0.0832	Avg Loss: 0.0000
Train Epoch: 41	[12000/28753 (42%)]	Total Loss: 0.0911	Avg Loss: 0.0000
Train Epoch: 41	[13000/28753 (45%)]	Total Loss: 0.0985	Avg Loss: 0.0000
Train Epoch: 41	[14000/28753 (49%)]	Total Loss: 0.1051	Avg Loss: 0.0000
Train Epoch: 41	[15000/28753 (52%)]	Total Loss: 0.1117	Avg Loss: 0.0000
Train Epoch: 41	[16000/28753 (56%)]	Total Loss: 0.1203	Avg Loss: 0.0000
Train Epoch: 41	[17000/28753 (59%)]	Total Loss: 0.1311	Avg Loss: 0.0000
Train Epoch: 41	[18000/28753 (62%)]	Total Loss: 0.1394	Avg Loss: 0.0000
Train Epoch: 41	[19000/28753 (66%)]	Total Loss: 0.1465	Avg Loss: 0.0000
Train Epoch: 41	[20000/28753 (69%)]	Total Loss: 0.1548	Avg Loss: 0.0000
Train Epoch: 41	[21000/28753 (73%)]	Total Loss: 0.1627	Avg Loss: 0.0000
Train Epoch: 41	[22000/28753 (76%)]	

In [10]:
# SAVE
models_folder = 'saved_models'
model_name = '20200903_setnet'
results_folder = 'saved_results'

In [11]:
# SAVE MODEL
torch.save(setnet, '%s/%s' % (models_folder, model_name))

In [13]:
# SAVE RESULTS
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
all_train_results_npy = np.array(all_train_results)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
all_val_results_npy = np.array(all_val_results)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
all_test_results_npy = np.array(all_test_results)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_results')), arr=all_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_results')), arr=all_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_results')), arr=all_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'fx_labels')), arr=fx_labels_npy)